## Importing Libraries

In [3]:
import pandas as pd
import numpy as np
import os

## Importing Data

In [5]:
# Create path
path = r'/Users/michele/Desktop/CF/files/Module4/Instacart/Data'

In [7]:
# Import orders_products_merged
df_ords_prods_merge = pd.read_pickle(os.path.join(path, 'Prepared', 'products_orders_merged.pkl'))

## Tasks

<b>Find the aggregated mean of the “order_number” column grouped by “department_id” for the entire dataframe.</b>

In [8]:
df_ords_prods_merge.groupby('department_id').agg({'order_number': ['mean']})

,order_number
,mean
department_id,
1,15.457838
2,17.277920
3,17.170395
4,17.811403
5,15.215751
6,16.439806
7,17.225802
8,15.340650


In [11]:
# Subset
df = df_ords_prods_merge[:1000000]

In [13]:
# Aggregated mean for the subset
df.groupby('department_id').agg({'order_number':['mean']})

,order_number
,mean
department_id,
1,14.800024
2,17.091743
3,17.913544
4,17.893092
5,15.214270
6,15.382135
7,17.694027
8,16.458105


 <b>How do the results for the entire dataframe differ from those of the subset?</b>

The average of the entire dataframe is slightly higher compared to the subset, but the difference is minimal

<b>Check the basic statistics of the product prices for each loyalty category (Loyal Customer, Regular Customer, and New Customer). Determine if the prices of products purchased by loyal customers differ from those purchased by regular or new customers.</b>

In [15]:
# Create max order column
df_ords_prods_merge['max_order'] = df_ords_prods_merge.groupby(['user_id'])['order_number'].transform('max')

In [17]:
# Flag the loyal customers
df_ords_prods_merge.loc[df_ords_prods_merge['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'

In [19]:
df_ords_prods_merge.loc[(df_ords_prods_merge['max_order'] <= 40) & (df_ords_prods_merge['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'

In [21]:
df_ords_prods_merge.loc[df_ords_prods_merge['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

In [23]:
# Check the basic statistics for the loyalty categories
df_ords_prods_merge.groupby('loyalty_flag').agg({'prices': ['mean','max','min']})

prices              
                       mean      max  min
loyalty_flag                             
Loyal customer    10.386336  99999.0  1.0
New customer      13.294670  99999.0  1.0
Regular customer  12.495717  99999.0  1.0

The minimum and maximum prices are the same for all the loyalty categories. The average price is higher for new customers and goes increasingly down for regular and loyal customers. 

In [25]:
df_ords_prods_merge.head()

,Unnamed: 0,order_id,user_id,order_number,order_days_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,_merge,product_name,aisle_id,department_id,prices,exists,max_order,loyalty_flag
0,0,2539329,1,1,2,8,0.0,196,1,0,both,Soda,77,7,9.0,both,10,New customer
1,0,2539329,1,1,2,8,0.0,14084,2,0,both,Organic Unsweetened Vanilla Almond Milk,91,16,12.5,both,10,New customer
2,0,2539329,1,1,2,8,0.0,12427,3,0,both,Original Beef Jerky,23,19,4.4,both,10,New customer
3,0,2539329,1,1,2,8,0.0,26088,4,0,both,Aged White Cheddar Popcorn,23,19,4.7,both,10,New customer
4,0,2539329,1,1,2,8,0.0,26405,5,0,both,XL Pick-A-Size Paper Towel Rolls,54,17,1.0,both,10,New customer


<b>Create a spending flag for each user based on the average price across all their orders using the following criteria:</b>

<b>If the mean of the prices of products purchased by a user is lower than 10, then flag them as a “Low spender.”</b>

<b>If the mean of the prices of products purchased by a user is higher than or equal to 10, then flag them as a “High spender.”</b>

In [31]:
# Create average spend column
df_ords_prods_merge['avg_spend'] = df_ords_prods_merge.groupby(['user_id'])['prices'].transform('mean')

In [33]:
# Flag the spending categories
df_ords_prods_merge.loc[df_ords_prods_merge['avg_spend'] <10,'Spending_flag'] = 'Low spender'
df_ords_prods_merge.loc[df_ords_prods_merge['avg_spend'] >=10,'Spending_flag'] = 'High spender'

In [35]:
# Frequency check
df_ords_prods_merge['Spending_flag'].value_counts(dropna = False)

Spending_flag
Low spender     31770614
High spender      634245
Name: count, dtype: int64

<b>Create an order frequency flag that marks the regularity of a user’s ordering behavior according to the median in the “days_since_prior_order” column</b>

<b>If the median of “days_since_prior_order” is higher than 20, then the customer should be labeled a “Non-frequent customer.”</b>

<b>If the median is higher than 10 and lower than or equal to 20, then the customer should be labeled a “Regular customer.”</b>

<b>If the median is lower than or equal to 10, then the customer should be labeled a “Frequent customer.”</b>

In [37]:
# Create median time column
df_ords_prods_merge['median_time'] = df_ords_prods_merge.groupby(['user_id'])['days_since_prior_order'].transform('median')

In [39]:
# Flag the customer's behaviour 
df_ords_prods_merge.loc[df_ords_prods_merge['median_time'] > 20, 'Regularity_flag'] = 'Non-frequent customer'
df_ords_prods_merge.loc[(df_ords_prods_merge['median_time'] > 10) & (df_ords_prods_merge['median_time'] <= 20), 'Regularity_flag'] = 'Regular customer'
df_ords_prods_merge.loc[df_ords_prods_merge['median_time'] <= 10, 'Regularity_flag'] = 'Frequent customer'

In [41]:
# Frequency check
df_ords_prods_merge['Regularity_flag'].value_counts(dropna = False)

Regularity_flag
Frequent customer        22796659
Regular customer          6921472
Non-frequent customer     2686728
Name: count, dtype: int64